<a href="https://colab.research.google.com/github/jeffersonramelo/Peper-relato-integrado-DIF-DIF/blob/main/Painel_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas openpyxl

In [6]:
import pandas as pd

# Caminho para o seu arquivo XLSX
arquivo_xlsx = '/content/merged_data_final todas.xlsx'

# Lendo o arquivo XLSX
try:
    df_panel = pd.read_excel(arquivo_xlsx, engine='openpyxl')

    # Excluindo linhas que contenham "Finanças e Seguros" na coluna "SetorEconomatica"
    df_filtrado = df_panel[df_panel['SetorEconomatica'] != 'Finanças e Seguros']

    # Exibindo as primeiras linhas do DataFrame filtrado para verificar os dados
    print(df_filtrado.head())
except Exception as e:
    print(f"Erro ao ler o arquivo: {e}")


   id codigo                                Nome_Companhia        Nome  \
0   1   AESB                        AES Brasil Energia S/A  AES Brasil   
1   2   AMBP  AMBIPAR PARTICIPAÇÕES E EMPREENDIMENTOS S.A.     Ambipar   
2   3   AZUL                                     AZUL S.A.        Azul   
3   4   CEAB                                C&A MODAS S.A.   Cea Modas   
4   5   SBSP        CIA SANEAMENTO BÁSICO ESTADO SÃO PAULO      Sabesp   

    SetorEconômicob3    SubsetorBovespa  ValorMercado30Abr11  \
0  Utilidade pública   Energia elétrica                  NaN   
1  Utilidade pública  Água e saneamento                  NaN   
2   Bens industriais         Transporte                  NaN   
3    Consumo cíclico           Comércio                  NaN   
4  Utilidade pública  Água e saneamento          10382514.91   

   ValorMercado30Mai11  ValorMercado30Jun11  PU30Abr11  ...  Setor_Outros  \
0                  NaN                  NaN        NaN  ...             0   
1               

In [7]:
from scipy.stats.mstats import winsorize

# Assumindo que df já foi definido e filtrado para os anos de 2020, 2021 e 2022
for col in ['PatrimLiqDez2010', 'LucroLiquidoDez2010']:
    # Aplicando winsorize em cada coluna selecionada com limites de 1% para os outliers em ambos os lados da distribuição
    df_panel[col] = winsorize(df_panel[col], limits=[0.01, 0.01])


In [11]:
print(df_panel.columns.tolist())


['codigo', 'Nome_Companhia', 'Nome', 'SetorEconômicob3', 'SubsetorBovespa', 'ValorMercado30Abr11', 'ValorMercado30Mai11', 'ValorMercado30Jun11', 'PU30Abr11', 'PU30Mai11', 'PU30Jun11', 'PatrimLiqDez2010', 'LucroLiquidoDez2010', 'ativototalt-1', 'ativototalt', 'IntangDez2010', 'CaixaEEqCxDez2010', 'ReceitaDez2010', 'CxGerOpDez2010', 'OuReAbDez2010', 'LPADez2010', 'quantacoes', 'Data_Entrega', 'Categoria', 'Link_Download', 'd_divulga', 'IIRC', 'financeiro', 'manufaturado', 'intelectual', 'humano', 'social', 'relacionamento', 'natural', 'd_IIRC', 'IIRC+ESG', 'IIRC+auditoria', 'SASB', 'CPC_09', 'value_reporting_foundation', 'asseguracao', 'Auditores_Independentes', 'GRI', 'ESG', 'Environmental', 'Social', 'Governance', 'Ambiental', 'Governança', 'fog_index', 'flesch_index', 'd_fog_divulga', 'd_flesch_divulga', 'd_fog_IIRC', 'd_flesch_IIRC', 'SetorEconomatica', 'Setor_Agro e Pesca', 'Setor_Alimentos e Beb', 'Setor_Comércio', 'Setor_Construção', 'Setor_Eletroeletrônicos', 'Setor_Energia Elétr

In [8]:
pip install linearmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 3.6 MB/s eta 0:00:00


In [13]:
# Supondo que df_panel seja seu DataFrame original
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects, PooledOLS
from linearmodels.panel import compare

# Corrigindo os nomes das variáveis baseado na lista fornecida
X_colunas = [
    'PatrimLiqDez2010', 'LucroLiquidoDez2010', 'Setor_Agro e Pesca', 'Setor_Alimentos e Beb',
    'Setor_Comércio', 'Setor_Construção', 'Setor_Eletroeletrônicos', 'Setor_Minerais não Met', 'Setor_Mineração', 'Setor_Máquinas Indust',
    'Setor_Outros', 'Setor_Papel e Celulose', 'Setor_Petróleo e Gas', 'Setor_Química',
    'Setor_Siderur & Metalur', 'Setor_Software e Dados', 'Setor_Telecomunicações', 'Setor_Textil',
    'Setor_Transporte Serviç', 'Setor_Veiculos e peças', 'd_divulga'
]

# Supondo que 'id' e 'ano' sejam colunas no DataFrame e não o índice MultiIndex neste ponto
# Se 'id' e 'ano' não estão como índice, isso configura eles como tal
if not isinstance(df_panel.index, pd.MultiIndex):
    df_panel = df_panel.set_index(['id', 'ano'])

# Verificar se a configuração foi realizada corretamente
print(df_panel.index)

# Preparar 'y' e 'X' com o DataFrame corrigido
y = df_panel['PU30Jun11']
X = df_panel[X_colunas]

# Adicionar a constante a 'X'
X = sm.add_constant(X)

# Ajustando os modelos
modelo_ef = PanelOLS(y, X, entity_effects=True).fit()
modelo_ea = RandomEffects(y, X).fit()
modelo_pooled = PooledOLS(y, X).fit()

# Comparando os modelos
resultado_comparacao = compare({"Efeitos Fixos": modelo_ef, "Efeitos Aleatórios": modelo_ea, "Pooled OLS": modelo_pooled})
print(resultado_comparacao)


MultiIndex([(  1, 2010),
            (  2, 2010),
            (  3, 2010),
            (  4, 2010),
            (  5, 2010),
            (  6, 2010),
            (  7, 2010),
            (  8, 2010),
            (  9, 2010),
            ( 10, 2010),
            ...
            (383, 2022),
            (384, 2022),
            (385, 2022),
            (385, 2022),
            (385, 2022),
            (386, 2022),
            (387, 2022),
            (388, 2022),
            (389, 2022),
            (407, 2022)],
           names=['id', 'ano'], length=5264)


/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


AbsorbingEffectError: 
The model cannot be estimated. The included effects have fully absorbed
one or more of the variables. This occurs when one or more of the dependent
variable is perfectly explained using the effects included in the model.

The following variables or variable combinations have been fully absorbed
or have become perfectly collinear after effects are removed:

          const, PatrimLiqDez2010, LucroLiquidoDez2010, Setor_Agro e Pesca, Setor_Alimentos e Beb, Setor_Comércio, Setor_Construção, Setor_Eletroeletrônicos, Setor_Minerais não Met, Setor_Mineração, Setor_Máquinas Indust, Setor_Outros, Setor_Papel e Celulose, Setor_Petróleo e Gas, Setor_Química, Setor_Siderur & Metalur, Setor_Software e Dados, Setor_Telecomunicações, Setor_Textil, Setor_Transporte Serviç, Setor_Veiculos e peças, d_divulga
          const, Setor_Agro e Pesca, Setor_Alimentos e Beb, Setor_Comércio, Setor_Construção, Setor_Eletroeletrônicos, Setor_Minerais não Met, Setor_Mineração, Setor_Máquinas 